In [ ]:
# Program extracting first column 
import xlrd

loc = ("./utils/Modified_Bioinfer_label.xlsx")

wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0) 
sheet.cell_value(0, 0) 
PPI = []

for i in range(1, sheet.nrows): 
    temp = []
    for j in range(sheet.ncols):
        temp.append(sheet.cell_value(i, j))
    PPI.append(temp)

## Vocabulary

In [ ]:
import nltk
for dependency in ("brown", "names", "wordnet", "averaged_perceptron_tagger", "universal_tagset", "punkt", "stopwords"):
    nltk.download(dependency)

In [ ]:
import string
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
punctuation = list(string.punctuation)
stopWords = set(stopwords.words('english'))

In [ ]:
def preprocess(sent):
    temp = word_tokenize(sent)
    filter = []
    for w in temp:
        if w.lower()not in stopWords and w not in punctuation:
            filter.append(w)
    return(' '.join(filter))

In [ ]:
corpus = []
corpus_raw = []
labels = []
proteins = []
for line in PPI:
    proteins.append(line[4:6])
    corpus.append(preprocess(line[1].lower()))
    corpus_raw.append(line[1].lower())
    labels.append(line[-1])

In [ ]:
vocab = []
vocab_raw = []
for doc in corpus:
    vocab += doc.split()
for doc in corpus_raw:
    vocab_raw += doc.split()

vocab = list(set(vocab))
vocab_raw = list(set(vocab_raw))
values = np.array(vocab)

In [ ]:
len(vocab_raw)

In [ ]:
len(vocab)

## Encode the Sentences / Documents

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded.size)

In [ ]:
def encode_sentence(sentence):
    indices = label_encoder.transform(sentence.split())
    vector = np.zeros(integer_encoded.size, dtype = 'int')
    for index in indices:
        vector[index] = 1
    return(vector)

In [ ]:
enc_corpus = []
for doc, label in tqdm(zip(corpus, labels), total = len(corpus)):
    enc_corpus.append(encode_sentence(doc))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1000, svd_solver='arpack')
red_dim_corpus = pca.fit_transform(enc_corpus)

# Node formatting

In [ ]:
index = 0
nodes_true = []
nodes_false = []

for doc, label in tqdm(zip(red_dim_corpus, labels), total = len(red_dim_corpus)):
    temp = ''
    temp += str(index) + '\t'
    temp += '\t'.join([str(n) for n in doc])
    temp += '\t'
    temp += '\t'+ str(float(label)) + '\n'
    index += 1
    if(label):
        nodes_true.append(temp)
    else:
        nodes_false.append(temp)

In [ ]:
from random import shuffle

nodes = []
nodes.extend(nodes_true)
nodes.extend(nodes_false)

shuffle(nodes)

In [ ]:
with open('./utils/node_text', 'w') as f:
    f.writelines(nodes)

# Edge Formatting

In [ ]:
protein_corpus = []
for doc_proteins in proteins:
    for protein in doc_proteins:
        protein_corpus.append(protein)
        
protein_corpus = list(set(protein_corpus))

In [ ]:
protein_encoder = LabelEncoder()
protein_encodings = protein_encoder.fit_transform(protein_corpus)

In [ ]:
doc_protein_encodings = []
for protein in proteins:  
    doc_protein_encodings.append(protein_encoder.transform(protein))

In [ ]:
edges = []
for i in tqdm(range(len(doc_protein_encodings))):
    for j, compare_to in enumerate(doc_protein_encodings):
        for compare in doc_protein_encodings[i]:
            if i != j and compare in compare_to:
                edges.append(str(i) + "\t" + str(j) + "\n")
                break

In [ ]:
len(edges)

In [ ]:
with open('./utils/link_text', 'w') as f:
    f.writelines(edges)